In [104]:
pwd


'C:\\Users\\Ramya\\Desktop\\Ramya\\GenomicDataScience\\Algorithms for GDS'

In [105]:
cd "C:\Users\Ramya\Desktop\Ramya\GenomicDataScience\Algorithms for GDS"

C:\Users\Ramya\Desktop\Ramya\GenomicDataScience\Algorithms for GDS


In [103]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome


In [106]:
genome = readGenome("lambda_virus.fa")

In [107]:
len(genome)

48502

In [108]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [296]:
def naive_with_rc(p, t):
    occurrences = [] 
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True     # 3 matches categorised - actual match, rc match and matches where the p and rc(p) is same
        rmatch = True
        smatch = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
            if t[i+j] != reverseComplement(p)[j]: # compare with the reverse complement
                rmatch = False
            if p[j] != reverseComplement(p)[j]:   # check to see if p is equal to its reerse complement
                smatch = False     
        if smatch:
            if match and rmatch:          # all 3 kinds of matches recorded
                occurrences.append(i)
        elif not smatch:
            if match:
                occurrences.append(i)
            elif rmatch:
                occurrences.append(i)        
    return occurrences

In [297]:
# Testing example
p = 'CCC'
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CCC' + ten_as + 'GGG' + ten_as
r = naive_with_rc(p, t)
print(r)

[10, 23]


In [298]:
# Testing example
p = 'CGCG'
t = ten_as + 'CGCG' + ten_as + 'CGCG' + ten_as
r = naive_with_rc(p, t)
print(r)

[10, 24]


In [303]:
# Testing example
phix_genome = readGenome('phix.fa')

In [311]:
# Testing example
occurrences = naive_with_rc('ATTA', phix_genome)

In [322]:
# Testing example
print('offset of leftmost occurrence: %d' % min(occurrences))

offset of leftmost occurrence: 10


In [313]:
# Testing example
print('# occurrences: %d' % len(occurrences))

# occurrences: 60


How many times does AGGT or its reverse complement (ACCT) occur in the lambda virus genome?
E.g. if AGGT occurs 10 times and ACCT occurs 12 times, you should report 22.

In [316]:
p = 'AGGT'
t = genome
r = naive_with_rc(p, t)
print(len(r))

306


How many times does TTAA or its reverse complement occur in the lambda virus genome?
Hint: TTAA and its reverse complement are equal, so remember not to double count.

In [317]:
p = 'TTAA'
t = genome
occurrences = naive_with_rc(p, t)
print(len(occurrences))

195


What is the offset of the leftmost occurrence of ACTAAGT or its reverse complement in the Lambda virus genome?
E.g. if the leftmost occurrence of ACTAAGT is at offset 40 (0-based) and the leftmost occurrence of its reverse 
complement ACTTAGT is at offset 29, then report 29.

In [319]:
p = 'ACTAAGT'
t = genome
occurrences = naive_with_rc(p, t)
print(min(occurrences))

26028


What is the offset of the leftmost occurrence of AGTCGA or its reverse complement in the Lambda virus genome?

In [320]:
p = 'AGTCGA'
t = genome
occurrences = naive_with_rc(p, t)
print(min(occurrences))

450


For Questions 5 and 6, make a new version of the naive function called naive_2mm that allows up to 2 mismatches per occurrence. 
Unlike for the previous questions, do not consider the reverse complement here. 
We are looking for approximate matches for P itself, not its reverse complement.
For example, ACTTTA occurs twice in ACTTACTTGATAAAGT, once at offset 0 with 2 mismatches, 
and once at offset 4 with 1 mismatch. So naive_2mm('ACTTTA', 'ACTTACTTGATAAAGT') should return the list [0, 4].

In [226]:
def naive_2mm(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = 0
        mm = 0
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                mm += 1
                if mm > 2: # allowing upto 2 mismatches
                    break
            else:
                match +=1
        if match >= (len(p)-2) and mm <= 2: # both mismatches and actual matches recorded
            if i not in occurrences:
                occurrences.append(i)
        if match == len(p):
            if i not in occurrences:
                occurrences.append(i)
                
    return occurrences

In [321]:
# Testing example
p = 'CTGT'
ten_as = 'AAAAAAAAAA'
t = ten_as + 'CTGT' + ten_as + 'CTTT' + ten_as + 'CGGG' + ten_as 
occurrences = naive_2mm(p, t)
print(occurrences)

[10, 24, 38]


In [228]:
# Testing example
phix_genome = readGenome('phix.fa')

In [229]:
# Testing example
occurrences = naive_2mm('GATTACA', phix_genome)

In [230]:
# Testing example
print('offset of leftmost occurrence: %d' % min(occurrences))

offset of leftmost occurrence: 10


In [231]:
# Testing example
print('# occurrences: %d' % len(occurrences))

# occurrences: 79


How many times does TTCAAGCC occur in the Lambda virus genome when allowing up to 2 mismatches?

In [232]:
p = 'TTCAAGCC'
t = genome
occurrences = naive_2mm(p, t)
print(len(occurrences))

191


What is the offset of the leftmost occurrence of AGGAGGTT in the Lambda virus genome when allowing up to 2 mismatches?

In [233]:
p = 'AGGAGGTT'
t = genome
occurrences = naive_2mm(p, t)
print(min(occurrences))

49


This dataset (ERR037900_1.first1000.fastq) has something wrong with it; one of the sequencing cycles is poor quality.

Report which sequencing cycle has the problem. Remember that a sequencing cycle corresponds
to a particular offset in all the reads. For example, if the leftmost read position seems to 
have a problem consistently across reads, report 0. If the fourth position from the left has the 
problem, report 3. Do whatever analysis you think is needed to identify the bad cycle. 
It might help to review the "Analyzing reads by position" video.

In [235]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [239]:
seqs, quals = readFastq("ERR037900_1.first1000.fastq")

In [240]:
def phred33toQ(qual):
    return ord(qual) - 33

In [291]:
def findErrpos(reads):
    errpos = {}
    for read in reads: # loop over the reads
        lowpos = []
        for i in range(len(read)): # loop through each character of a read for lowest Q
            if phred33toQ(read[i]) == 2:
                lowpos.append(i)   # Append list for position of low value after each read
        for p in lowpos:
            if p not in errpos:  # Append dictionary with position and count from the list
                errpos[p] = 1
            else:
                errpos[p] += 1
    return errpos         

In [292]:
sc = findErrpos(quals)

In [294]:
max(sc.values())

919

In [295]:
sc

{33: 1,
 34: 2,
 35: 5,
 36: 5,
 37: 5,
 38: 6,
 39: 10,
 40: 13,
 41: 13,
 42: 13,
 43: 14,
 44: 19,
 45: 23,
 46: 25,
 47: 28,
 48: 29,
 49: 34,
 50: 49,
 51: 50,
 52: 60,
 53: 56,
 54: 64,
 55: 86,
 56: 90,
 57: 93,
 58: 97,
 59: 106,
 60: 142,
 61: 144,
 62: 152,
 63: 156,
 64: 184,
 65: 284,
 66: 919,
 67: 287,
 68: 290,
 69: 293,
 70: 303,
 71: 303,
 72: 303,
 73: 305,
 74: 309,
 75: 313,
 76: 314,
 77: 318,
 78: 329,
 79: 346,
 80: 390,
 81: 392,
 82: 398,
 83: 409,
 84: 431,
 85: 473,
 86: 484,
 87: 497,
 88: 512,
 89: 534,
 90: 572,
 91: 576,
 92: 583,
 93: 591,
 94: 613,
 95: 644,
 96: 649,
 97: 654,
 98: 661,
 99: 670}